In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import os
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from math import sqrt
import datetime
import pickle

import sklearn
from sklearn import preprocessing
from sklearn import linear_model

In [ ]:
%matplotlib inline

# Load the preprocessed data

In [ ]:
# Get the pre-processed data:
X = pickle.load(open('../data/fs_data_lags_1_12.pkl', 'rb'))

In [ ]:
X.columns

# Category Encoding

In [8]:
# encode categorical variables (except item, because there are loads of them, where we rely on mean encoding, which we did in data_prep.py)
def category_encoding(fs_data):

    item_category_encoder             = preprocessing.OneHotEncoder(sparse=False)
    encoded_item_category_id          = pd.DataFrame(data=item_category_encoder.fit_transform(fs_data[['item_category_id']])) 
    encoded_item_category_id.columns  = [str(col) + '_item_category' for col in encoded_item_category_id.columns]
    
    shop_encoder            = preprocessing.OneHotEncoder(sparse=False)
    encoded_shop_id         = pd.DataFrame(data=shop_encoder.fit_transform(fs_data[['shop_id']]))
    encoded_shop_id.columns = [str(col) + '_shop' for col in encoded_shop_id.columns]
    
    # join the encoded versions to the main df
    fs_data = pd.concat([fs_data.reset_index(), encoded_item_category_id, encoded_shop_id], axis=1)

    return(fs_data.reset_index())

In [9]:
X = category_encoding(X)

In [10]:
# convenient to be able to refer to target and predictor values like this
target_var = 'item_cnt_month'
predictor_cols = [col for col in X.columns.tolist() if col not in [target_var, 'level_0', 'index', 'ID', 'item_name', 'shop_name',
                                                                   'date_block_num','item_category_name']]

In [11]:
predictor_cols

['item_category_id',
 'item_id',
 'shop_id',
 'ITEM_CATEGORY_mean_item_cnt_day_lag1',
 'ITEM_CATEGORY_mean_item_price_lag1',
 'ITEM_CATEGORY_sum_item_cnt_day_lag1',
 'ITEM_mean_item_cnt_day_lag1',
 'ITEM_mean_item_price_lag1',
 'ITEM_sum_item_cnt_day_lag1',
 'SHOP_mean_item_cnt_day_lag1',
 'SHOP_mean_item_price_lag1',
 'SHOP_sum_item_cnt_day_lag1',
 'item_cnt_month_lag1',
 'item_price_lag1',
 'ITEM_CATEGORY_mean_item_cnt_day_lag12',
 'ITEM_CATEGORY_mean_item_price_lag12',
 'ITEM_CATEGORY_sum_item_cnt_day_lag12',
 'ITEM_mean_item_cnt_day_lag12',
 'ITEM_mean_item_price_lag12',
 'ITEM_sum_item_cnt_day_lag12',
 'SHOP_mean_item_cnt_day_lag12',
 'SHOP_mean_item_price_lag12',
 'SHOP_sum_item_cnt_day_lag12',
 'item_cnt_month_lag12',
 'item_price_lag12',
 'item_name_cluster',
 'shop_already_seen',
 'item_already_seen',
 'item_cnt_month_lag1_lag12_diff',
 'ITEM_mean_item_cnt_day_lag1_lag12_diff',
 'SHOP_mean_item_cnt_day_lag1_lag12_diff',
 'item_cnt_month_lag1sq_lag12_diff',
 '0_item_category',


# Create test and validation sets

In [9]:
# Instead of creating separate train and validation data sets, we'll just partition the main set, 
# splitting based on time to match the way the test set works
train_rows      = np.where(X['date_block_num'] <= 32)[0]
validation_rows = np.where(X['date_block_num'] == 33)[0]
test_rows       = np.where(X['date_block_num'] == 34)[0]


# Fit model

In [10]:
fs_lm = linear_model.LinearRegression()

In [11]:
fs_lm.fit(X.iloc[train_rows][predictor_cols], X.iloc[train_rows][target_var],)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [12]:
# make predictions on validation set
preds = fs_lm.predict(X.iloc[validation_rows][predictor_cols])

In [13]:
# clip to 0-20 to match test data
preds = np.clip(preds, 0, 20)

# compare to the (similarly clipped) validation data
rmse = sqrt(mean_squared_error(preds, np.clip(X.iloc[validation_rows][target_var], 0, 20)))
rmse

0.9880384701717031

In [14]:
# fit a model with the validation rows as well
train_rows = np.where(X['date_block_num'] <= 33)[0]
fs_lm.fit(X.iloc[train_rows][predictor_cols].values, X.iloc[train_rows][target_var].values)  


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [19]:
# make predictions on validation set
preds = fs_lm.predict(X.iloc[test_rows][predictor_cols])

# clip to 0-20 to match test data
preds = np.clip(preds, 0, 20)

In [12]:
# save the predictions
pickle.dump(preds, open('fs_lm_lags_1_12_preds_up_to_month_34.pkl', 'wb'))


NameError: name 'preds' is not defined

In [13]:
# we seem to need to save this as well
pickle.dump(predictor_cols, open('fs_lm_lags_1_12_predictor_cols.pkl', 'wb'))

In [ ]:
# save the model
pickle.dump(fs_lm, open('fs_lm_lags_1_12.pkl', 'wb'))
